In [27]:
import pandas as pd
from dotenv import load_dotenv
import os
import time
from google.oauth2 import service_account
from googleapiclient.discovery import build

load_dotenv()

gtm_id = os.environ["GTM_ACCOUNT_ID"]

# Excel

In [28]:
xlsx_names = ['kyobo', 'library', 'scholar', 'story', 'tree']
service_names = {
    '교보문고_renewal': 'kyobo',
    '전자도서관': 'library',
    '스콜라': 'scholar',
    '스토리': 'story',
    '리딩트리': 'tree'
}

def get_xlsx():
    res = {}
    for name in service_names.values():
        res[name]=pd.ExcelFile(f'./xlsx/{name}.xlsx')

    return res

def get_sheet_name(excel):
    sheet_names = excel.sheet_names
    return sheet_names

def read_sheet(excel_file, sheet_name):
    return pd.read_excel(excel_file, sheet_name=sheet_name)

# GTM

In [29]:
# Get credentials
def get_credentials():
    SERVICE_ACCOUNT_FILE = './xlsx/gtm-api-409502-6f8362f190d7.json'

    credentials_container = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE,
        scopes=["https://www.googleapis.com/auth/tagmanager.edit.containers"]
    )

    return credentials_container

credentials = get_credentials()

# Get service
def get_service():
    service = build('tagmanager', 'v2', credentials = credentials)

    return service

service = get_service()

# Get container
def get_container():
    parent = f"accounts/{gtm_id}"

    containers = service.accounts().containers().list(
        parent=parent,
    ).execute()

    return containers['container']

container_origin = get_container()
containers = {}
for i in container_origin:
    containers[i['name']] = i


# Set Container

In [30]:
# 교보문고_renewal, 전자도서관, 스콜라, 스토리, 리딩트리
service_name = '교보문고_renewal'
container = containers[service_name]

xlsxs = get_xlsx()
xlsx = xlsxs[service_names[service_name]]

# Get workspace
def get_workspaces():
    parent=container['path']

    workspace = service.accounts().containers().workspaces().list(
        parent=parent
    ).execute()

    return workspace['workspace']

workspace_origin = get_workspaces()

workspaces = {}
for i in workspace_origin:
    workspaces[i['name']] = i

workspace = workspaces['new']

# Get Elements

In [31]:
def get_variables():
    parent=workspace['path']

    containers = service.accounts().containers().workspaces().variables().list(
        parent=parent
    ).execute()

    return containers

# Variables

In [7]:
def create_gtm_variable(request_body):
    parent=workspace['path']

    try:
        response = service.accounts().containers().workspaces().variables().create(
            parent=parent,
            body=request_body
        ).execute()

        print(f"Created variable: {response['name']}")
        return response
    except Exception as e:
        print(f"An error occurred: {e}")
        print(f"Above error occurred with: {request_body['name']}")
        return e

def list_gtm_variable():
    parent=workspace['path']

    try:
        response = service.accounts().containers().workspaces().variables().list(
            parent=parent
        ).execute()

        print(f"Listed variables: {response['variable']}")
        return response
    except Exception as e:
        print(f"An error occurred: {e}")

In [96]:
def create_aev_variable(sheet):
    error = 0
    for row in sheet.iloc():
        if (row[1] == '요소 속성'):
            parameter = [
                {
                    "type": "template",
                    "key": "elementSelector",
                    "value": "ATTRIBUTE"
                },
                {
                    'type': 'template',
                    'key': 'attribute',
                    'value': row[2]
                },
                {
                    'type': 'boolean',
                    'key': 'setDefaultValue',
                    'value': 'false'
                },
            ]

        request_body = {
            "name": row[0],
            "type": 'aev',
            "parameter": parameter
        }

        try:
            create_gtm_variable(request_body)
            time.sleep(5)
        except Exception as e:
            error += 1

    print(f"Total Error: {error}")

In [97]:
def create_dom_variable(sheet):
    error = 0
    for row in sheet.iloc():
        if (row[1] == 'CSS 선택도구'):
            parameter = [
                {
                    "type": "template",
                    "key": "elementSelector",
                    "value":row[2] 
                },
                {
                    'type': 'template',
                    'key': 'selectorType',
                    'value': 'CSS'
                }
            ]

        request_body = {
            "name": row[0],
            "type": 'd',
            "parameter": parameter
        }

        try:
            create_gtm_variable(request_body)
            time.sleep(5)
        except Exception as e:
            error += 1

    print(f"Total Error: {error}")

In [107]:
def create_table_variable(sheet):
    error = 0
    request_body = {}

    for row in sheet.iloc():
        if (type(row[0]) is str):
            if (len(request_body) != 0):
                try:
                    create_gtm_variable(request_body)
                    time.sleep(5)
                except Exception as e:
                    error += 1

            request_body['name'] = row[0]
            formatValue = {}
            parameter = []
            map_data = []
            parameter.append({
                'type': 'list',
                'key': 'map',
                'list': map_data
            })
            request_body['parameter'] = parameter

            if (row[1] == '참고표'):
                request_body['type'] = 'smm'
            if (row[1] == '정규식 표'):
                request_body['type'] = 'remm'
                request_body['parameter'].append({'type': 'boolean', 'key': 'fullMatch', 'value': 'true'})
                request_body['parameter'].append({'type': 'boolean', 'key': 'replaceAfterMatch', 'value': 'true'})
                request_body['parameter'].append({'type': 'boolean', 'key': 'ignoreCase', 'value': 'true'})

        if (type(row[5]) is str):
            parameter.append({
                'type': 'boolean',
                'key': 'setDefaultValue',
                'value': 'true'
            })
            parameter.append({
                'type': 'template',
                'key': 'defaultValue',
                'value': row[6]
            })

        if (type(row[7]) is str):
            if (row[7][:5] == 'undef'):
                formatValue['convertUndefinedToValue'] = {
                    'type': 'template',
                    'value': row[8]
                }
            if (row[7][:5] == 'false'):
                formatValue['convertFalseToValue'] = {
                    'type': 'template',
                    'value': row[8]
                }
            if (row[7][:4] == 'null'):
                formatValue['convertNullToValue'] = {
                    'type': 'template',
                    'value': row[8]
                }
            

        if (type(row[2]) is str):
            parameter.append({
                'type': 'template',
                'key': 'input',
                'value': row[2]
            })

        if (type(row[3]) is str):
            input_data = {
                'type': 'map',
                'map': [
                    {
                        'type': 'template',
                        'key': 'key',
                        'value': row[3]
                    },
                    {
                        'type': 'template',
                        'key': 'value',
                        'value': row[4]
                    }
                ]
            }
            map_data.append(input_data)

        
        request_body['formatValue'] = formatValue

    try:
        create_gtm_variable(request_body)
        time.sleep(5)
    except Exception as e:
        error += 1

    print(f"Total Error: {error}")

In [83]:
def create_gtes_variable(sheet):
    error = 0
    request_body = {}
    for row in sheet.iloc():
        if (type(row[0]) is str):
            if (len(request_body) != 0):
                try:
                    create_gtm_variable(request_body)
                    time.sleep(5)
                except Exception as e:
                    error += 1

            request_body['name'] = row[0]
            request_body['type'] = 'gtes'
            eventSettingsTable = []
            userProperties = []

        if (type(row[2]) is str):
            event_data = {
                'type': 'map',
                'map': [
                    {
                        'type': 'template',
                        'key': 'parameter',
                        'value': row[2]
                    },
                    {
                        'type': 'template',
                        'key': 'parameterValue',
                        'value': row[3]
                    }
                ]
            }
            eventSettingsTable.append(event_data)
        if (type(row[4]) is str):
            user_data = {
                'type': 'map',
                'map': [
                    {
                        'type': 'template',
                        'key': 'name',
                        'value': row[4]
                    },
                    {
                        'type': 'template',
                        'key': 'value',
                        'value': row[5]
                    }
                ]
            }
            userProperties.append(user_data)

        request_body['parameter'] = [{
            'type': 'list',
            'key': 'eventSettingsTable',
            'list': eventSettingsTable
        },
        {
            'type': 'list',
            'key': 'userProperties',
            'list': userProperties
        }]

    try:
        create_gtm_variable(request_body)
        time.sleep(5)
    except Exception as e:
        error += 1

    print(f"Total Error: {error}")

In [84]:
def create_url_variable(sheet):
    error = 0
    for row in sheet.iloc():
        if (row[2] == '검색어'):
            parameter = [
                {
                "type": "template",
                "key": "component",
                "value": "QUERY"
                },
                {
                    'type': 'template',
                    'key': 'queryKey',
                    'value': row[3]
                }
            ]
            type = 'u'

        request_body = {
            "name": row[0],
            "type": type,
            "parameter": parameter
        }

        try:
            create_gtm_variable(request_body)
            time.sleep(5)
        except Exception as e:
            error += 1

    print(f"Total Error: {error}")

In [85]:
def create_dataLayer_variable(sheet):
    error = 0
    for row in sheet.iloc():
        if (row[1] == '상수'):
            parameter = [{
                'type': 'template',
                'key': 'value',
                'value': row[2],
            }]
            type = 'c'
        elif (row[1] == '맞춤 자바스크립트'):
            parameter = [{
                'type': 'template',
                'key': 'javascript',
                'value': row[2],
            }]
            type = 'jsm'
        elif (row[1] == '당사 쿠키'):
            parameter = [{
                'type': 'template',
                'key': 'name',
                'value': row[2],
            }]
            type = 'k'
        else:
            parameter = [
                {
                    'type': 'template',
                    'key': 'name',
                    'value': row[2],
                },
                {
                    'type': 'integer',
                    'key': 'dataLayerVersion',
                    'value': 2,
                }
                ]
            type = 'v'

        request_body = {
            'name': row[0],
            'type': type,
            'parameter': parameter,
        }

        try:
            create_gtm_variable(request_body)
            time.sleep(5)
        except Exception as e:
            error += 1

    print(f"Error: {error}")


In [182]:
sheet_names = get_sheet_name(xlsx)

for index, i in enumerate(sheet_names):
  print(f"{index}: {i}")

0: 변수(당사쿠키)
1: 변수(데이터 영역)
2: 변수(URL)
3: 변수(자동이벤트 변수)
4: 변수(DOM 요소)
5: 변수(맞춤JS)
6: 변수(참고표)
7: 변수(이벤트 매개변수)
8: 트리거(맞춤 이벤트)
9: 트리거(페이지뷰)
10: 트리거(스크롤 깊이)
11: 트리거(링크 클릭)(Web)
12: 트리거(링크 클릭)(App)
13: 트리거(모든 요소 클릭)(Web)
14: 트리거(모든 요소 클릭)(App)
15: 태그(맞춤HTML)
16:  태그(Web)
17:  태그(App 이벤트)


In [108]:
# sheet_constants = read_sheet(xlsx, sheet_names[0])
# sheet_cookie = read_sheet(xlsx, sheet_names[1])
# sheet_data_layer = read_sheet(xlsx, sheet_names[1])
# sheet_url = read_sheet(xlsx, sheet_names[1])
# sheet_aev = read_sheet(xlsx, sheet_names[4])
# sheet_dom = read_sheet(xlsx, sheet_names[2])
# sheet_custom_js = read_sheet(xlsx, sheet_names[3])
sheet_table = read_sheet(xlsx, sheet_names[6])
# sheet_gtes = read_sheet(xlsx, sheet_names[5])

# create_dataLayer_variable(sheet_constants)
# create_dataLayer_variable(sheet_cookie)
# create_dataLayer_variable(sheet_data_layer)
# create_dataLayer_variable(sheet_custom_js)
# create_url_variable(sheet_url)
# create_aev_variable(sheet_aev)
# create_dom_variable(sheet_dom)
# create_gtes_variable(sheet_gtes)
create_table_variable(sheet_table)

/var/folders/xq/qf2k1k454qb_jfv62h4qfhn00000gn/T/ipykernel_45950/3354141961.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (type(row[0]) is str):
/var/folders/xq/qf2k1k454qb_jfv62h4qfhn00000gn/T/ipykernel_45950/3354141961.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  request_body['name'] = row[0]
/var/folders/xq/qf2k1k454qb_jfv62h4qfhn00000gn/T/ipykernel_45950/3354141961.py:25: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[po

An error occurred: <HttpError 400 when requesting https://tagmanager.googleapis.com/tagmanager/v2/accounts/6003640738/containers/90556056/workspaces/43/variables?alt=json returned "Returned an error response for your request.". Details: "[{'message': 'Returned an error response for your request.', 'domain': 'global', 'reason': 'badRequest'}]">
Above error occurred with: TD - item_list_id
An error occurred: <HttpError 400 when requesting https://tagmanager.googleapis.com/tagmanager/v2/accounts/6003640738/containers/90556056/workspaces/43/variables?alt=json returned "Returned an error response for your request.". Details: "[{'message': 'Returned an error response for your request.', 'domain': 'global', 'reason': 'badRequest'}]">
Above error occurred with: TD - item_list_name
An error occurred: <HttpError 400 when requesting https://tagmanager.googleapis.com/tagmanager/v2/accounts/6003640738/containers/90556056/workspaces/43/variables?alt=json returned "Returned an error response for your

/var/folders/xq/qf2k1k454qb_jfv62h4qfhn00000gn/T/ipykernel_45950/3354141961.py:77: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'value': row[3]
/var/folders/xq/qf2k1k454qb_jfv62h4qfhn00000gn/T/ipykernel_45950/3354141961.py:82: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'value': row[4]


Created variable: TD - 메뉴 - 서비스탭 명칭
Created variable: TD - 검색 팝업 - 링크 이름
Created variable: TD - 검색 팝업 - 버튼 이름
Created variable: TD - gbCode 명칭
Created variable: TD - target 명칭
Created variable: TD - 상품 상세 공유 수단


/var/folders/xq/qf2k1k454qb_jfv62h4qfhn00000gn/T/ipykernel_45950/3354141961.py:54: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'value': row[8]


An error occurred: <HttpError 400 when requesting https://tagmanager.googleapis.com/tagmanager/v2/accounts/6003640738/containers/90556056/workspaces/43/variables?alt=json returned "Returned an error response for your request.". Details: "[{'message': 'Returned an error response for your request.', 'domain': 'global', 'reason': 'badRequest'}]">
Above error occurred with: TD - 운영/개발 서버 구분
Created variable: TD - item_list_tab(PICKS)
An error occurred: <HttpError 400 when requesting https://tagmanager.googleapis.com/tagmanager/v2/accounts/6003640738/containers/90556056/workspaces/43/variables?alt=json returned "Returned an error response for your request.". Details: "[{'message': 'Returned an error response for your request.', 'domain': 'global', 'reason': 'badRequest'}]">
Above error occurred with: TD - 마이룸 - item_list_name
An error occurred: <HttpError 400 when requesting https://tagmanager.googleapis.com/tagmanager/v2/accounts/6003640738/containers/90556056/workspaces/43/variables?alt=j